## Практическая работа

В этой практической работе четыре обязательные задачи.

*Обязательные задачи* нужно сделать для того, чтобы проверить, что вы действительно усвоили материал модуля. Сдайте их на проверку.

Удачи!

Цели практической работы: 


1.   Потренироваться в обучении моделей деревьев решений.
2.   Потренироваться в обучении моделей случайного леса. 
3.   Научиться оценивать качество моделей с помошью Accuracy и confusion matrix.
4.   Научиться увеличивать качество моделей с помощью тюнинга параметров.




Что оценивается:

*   Все пункты и критерия приёмки задания выполнены.
*   Удаление колонок по результатам feature_importance и определения типов производится с помощью кода, а не перечислением их вручную.




## Обязательные задачи

### Описание датасета:
- `id`: идентификатор записи;
- `is_manufacturer_name`: признак производителя автомобиля;

- `region_*`: регион;
- `x0_*`: тип топлива;
- `manufacturer_*`: производитель;
- `short_model_*`: сокращённая модель автомобиля;
- `title_status_*`: статус;
- `transmission_*`: коробка передач;
- `state_*`: штат;
- `age_category_*`: возрастная категория автомобиля;

- `std_scaled_odometer`: количество пройденных миль (после стандартизации);
- `year_std`: год выпуска (после стандартизации);
- `lat_std`: широта (после стандартизации);
- `long_std`: долгота (после стандартизации);
- `odometer/price_std`: отношение стоимости к пробегу автомобиля (после стандартизации);
- `desc_len_std`: количество символов в тексте объявления о продаже (после стандартизации);
- `model_in_desc_std`: количество наименований модели автомобиля в тексте объявления о продаже (после стандартизации);
- `model_len_std`: длина наименования автомобиля (после стандартизации);
- `model_word_count_std`: количество слов в наименовании автомобиля (после стандартизации);
- `month_std`: номер месяца размещения объявления о продаже автомобиля (после стандартизации);
- `dayofweek_std`: день недели размещения объявления о продаже автомобиля (после стандартизации);
- `diff_years_std`: количество лет между годом производства автомобиля и годом размещения объявления о продаже автомобиля (после стандартизации);

- `price`: стоимость;
- `price_category`: категория цены.

0. *Подготовка базовой модели*

Обучите простую модель классификации с помощью DecisionTreeClassifier на данных из датасета vehicles_dataset_prepared.csv. Для этого сделайте следующие шаги:

1. Обучите модель дерева решений с зафиксированным random_state на тренировочной выборке.
2. Сделайте предикт на тестовой выборке.
3. Замерьте точность на тестовой выборке и выведите матрицу ошибок. 
4. Удалите фичи с нулевыми весами по feature_importance из тренировочной и тестовой выборок.
5. Заново обучите модель и измерьте качество.

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score, confusion_matrix 

In [2]:
df = pd.read_csv('data/vehicles_dataset_prepared.csv')

df_prepared = df.copy()
df_prepared = df_prepared.drop(['price', 'odometer/price_std'], axis=1)

x = df_prepared.drop(['price_category'], axis=1)
y = df_prepared['price_category']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [3]:
# Ваш код здесь 
dtc = DecisionTreeClassifier()
dtc.fit(x_train, y_train)

DecisionTreeClassifier()

In [4]:
dtc_pred = dtc.predict(x_test)

In [5]:
accuracy_score(y_test, dtc_pred)

0.668052668052668

In [6]:
confusion_matrix(y_test, dtc_pred)

array([[727,  54, 216],
       [ 42, 674, 237],
       [194, 215, 527]], dtype=int64)

In [7]:
f_imp_list = list(zip(x_train.columns, dtc.feature_importances_))
f_imp_list.sort(key=lambda x: x[1], reverse=True)

In [8]:
f_imp_list

[('diff_years_std', 0.2265174019849226),
 ('std_scaled_odometer', 0.0798244043663201),
 ('model_len_std', 0.07480199680204669),
 ('desc_len_std', 0.07083233641727633),
 ('lat_std', 0.04249456469342468),
 ('long_std', 0.03817017322848683),
 ('x0_diesel', 0.03339503900990173),
 ('id', 0.02893481541674513),
 ('year_std', 0.016932622022181907),
 ('dayofweek_std', 0.014057232646043614),
 ('manufacturer_kia', 0.009160285070398509),
 ('short_model_wrangler', 0.0088467771796998),
 ('manufacturer_ram', 0.008750927873828834),
 ('manufacturer_hyundai', 0.006220780364401149),
 ('model_word_count_std', 0.006195592873863728),
 ('short_model_f-150', 0.0058986228948152385),
 ('manufacturer_mercedes-benz', 0.0056831840293769505),
 ('short_model_silverado', 0.0055749414527343924),
 ('manufacturer_volkswagen', 0.005439814100151904),
 ('manufacturer_gmc', 0.005418918053307775),
 ('is_chevrolet', 0.004884721865257739),
 ('short_model_Scion', 0.004865746378858321),
 ('short_model_tacoma', 0.0043120026934051

In [9]:
important_features = [feature for feature, importance in f_imp_list if importance > 0.0]

x_df = df_prepared[important_features]
x_df

,diff_years_std,std_scaled_odometer,model_len_std,desc_len_std,lat_std,long_std,x0_diesel,id,year_std,dayofweek_std,...,region_greenville / upstate,state_wa,short_model_leaf,region_pueblo,region_kalispell,region_salem,region_phoenix,region_sacramento,"region_st louis, MO",region_central NJ
0,-1.322394,-1.079390,1.163032,0.632075,-0.619172,0.484245,1.0,7308295377,1.322394,1.120284,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.695973,-0.560115,0.932087,-0.646781,0.388014,1.110800,0.0,7316380095,0.695973,-1.374972,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.852578,-1.292863,0.470197,0.560744,-0.843059,0.531185,0.0,7313733749,0.852578,-0.376870,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.226157,0.352621,-0.915473,0.180435,-0.504509,0.853562,0.0,7308210929,0.226157,1.120284,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.069552,-0.234311,1.163032,0.766366,-0.460101,0.557607,0.0,7303797340,0.069552,0.122182,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9614,1.496501,0.873936,-0.915473,-0.376744,0.615626,0.823646,0.0,7304876387,-1.496501,1.120284,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9615,0.556869,0.293363,-0.453583,-0.654060,0.806702,0.903947,0.0,7316152972,-0.556869,-1.374972,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9616,0.087054,0.876868,1.509450,-0.668253,1.105428,-1.628875,0.0,7310993818,-0.087054,0.122182,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
9617,-0.539367,-0.529514,-0.569055,0.327100,0.773939,0.251959,0.0,7306637427,0.539367,-0.376870,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
X = x_df
Y = df_prepared['price_category']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [11]:
dtc2 = DecisionTreeClassifier()
dtc2.fit(X_train, Y_train)

DecisionTreeClassifier()

In [12]:
dtc2_pred = dtc2.predict(X_test)

In [13]:
accuracy_score(Y_test, dtc2_pred)

0.6729036729036729

In [14]:
confusion_matrix(Y_test, dtc2_pred)

array([[738,  57, 202],
       [ 43, 682, 228],
       [195, 219, 522]], dtype=int64)

In [15]:
f_imp_list2 = list(zip(X_train.columns, dtc2.feature_importances_))
f_imp_list2.sort(key=lambda x: x[1], reverse=True)

In [16]:
f_imp_list2

[('diff_years_std', 0.22690482047814792),
 ('std_scaled_odometer', 0.07965468784297153),
 ('model_len_std', 0.07762774220677054),
 ('desc_len_std', 0.07048922335035471),
 ('lat_std', 0.042989914219894745),
 ('long_std', 0.04020975855287283),
 ('x0_diesel', 0.033927656636615025),
 ('id', 0.03296939080037722),
 ('year_std', 0.016810411309143115),
 ('dayofweek_std', 0.015435252582964928),
 ('manufacturer_kia', 0.010029274024167149),
 ('manufacturer_ram', 0.00930564979716938),
 ('short_model_wrangler', 0.008817218605738029),
 ('model_word_count_std', 0.00773054355245593),
 ('short_model_f-150', 0.006313878562102587),
 ('manufacturer_hyundai', 0.006042876609772031),
 ('short_model_silverado', 0.005540691957165705),
 ('manufacturer_mercedes-benz', 0.0053794985404614784),
 ('model_in_desc_std', 0.005295882793631954),
 ('short_model_Scion', 0.004865746378858322),
 ('manufacturer_gmc', 0.004671791053133589),
 ('manufacturer_volkswagen', 0.004643874758632029),
 ('manufacturer_dodge', 0.004526432

1. *Подготовка модели случайного леса*

Обучите простую модель классификации с помощью RandomForestClassifier. Для этого сделайте следующие шаги:
1. На новых урезанных семплах тренировочной и тестовой выборок обучите модель случайного леса с зафиксированным random_state=50. 

2. Сделайте предикт и посчитайте точность модели и матрицу ошибок. Сравните с предыдущей моделью дерева решений. Есть ли случаи, когда модель из пункта 1 отрабатывает лучше, чем модель случайного леса?

In [17]:
# Ваш код здесь 
X = x_df
Y = df_prepared['price_category']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=50)

In [18]:
rfc = RandomForestClassifier()
rfc.fit(X_train, Y_train)

RandomForestClassifier()

In [19]:
y_pred = rfc.predict(X_test)

In [20]:
accuracy_score(Y_test, y_pred)

0.7543312543312544

In [21]:
confusion_matrix(Y_test, y_pred)

array([[785,  38, 123],
       [ 12, 811, 152],
       [163, 221, 581]], dtype=int64)

Думаю модель Деревьев решений лучше только в том плане что работает быстрее. но в любом другом аспекте Случайные леса гораздо лучше. Особенно в том плане что модель случайных лесов не может переобучиться

2. *Тюнинг модели случайного леса*

Увеличьте точность модели на тестовом датасете RandomForestClassifier c помощью тюнинга параметров. 

Параметры, которые можно настраивать для увеличения точности:

```
    `bootstrap'
    'max_depth'
    'max_features'
    'min_samples_leaf'
    'min_samples_split'
    'random_state'
    'n_estimators'

```



С описанием каждого из параметров можно ознакомиться в документации:


https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

Задание засчитывается, если значение метрики строго выше 0,76 на тестовом датасете.

In [22]:
# Ваш код здесь 
from sklearn.model_selection import RandomizedSearchCV

In [23]:
n_estimators = list(range(200, 601, 50))
max_features = ['sqrt', 'log2']
bootstrap = [True,False]
min_samples_leaf = list(range(1, 11))
min_samples_split = [2, 3, 4]

In [24]:
param_grid = { 'n_estimators':n_estimators,
             'max_features':max_features,
             'bootstrap':bootstrap,
             'min_samples_leaf': min_samples_leaf,
             'min_samples_split': min_samples_split } 

In [25]:
rfc2 = RandomForestClassifier()
randomized_search = RandomizedSearchCV(
   estimator=rfc2,
   param_distributions=param_grid,
   n_iter=20,
   scoring='accuracy',
   verbose=1,
   n_jobs=-1
)

In [26]:
randomized_search.fit(X_train, Y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


RandomizedSearchCV(estimator=RandomForestClassifier(), n_iter=20, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_features': ['sqrt', 'log2'],
                                        'min_samples_leaf': [1, 2, 3, 4, 5, 6,
                                                             7, 8, 9, 10],
                                        'min_samples_split': [2, 3, 4],
                                        'n_estimators': [200, 250, 300, 350,
                                                         400, 450, 500, 550,
                                                         600]},
                   scoring='accuracy', verbose=1)

In [27]:
randomized_search.best_params_

{'n_estimators': 600,
 'min_samples_split': 3,
 'min_samples_leaf': 1,
 'max_features': 'log2',
 'bootstrap': False}

In [28]:
predictions = randomized_search.predict(X_test)

In [29]:
accuracy_score(Y_test, predictions)

0.7726957726957727

In [30]:
confusion_matrix(Y_test, predictions)

array([[791,  35, 120],
       [ 12, 815, 148],
       [139, 202, 624]], dtype=int64)

3. *Анализ влияния фичей на модель*

Во всех задачах до вы работали над подготовленным датасетом, где фичи были заранее извлечены из текстовых переменных, отскейлены и пропущены через One Hot Encoder. Сравним, какой была бы предсказательная способность модели, если бы мы использовали только сырые данные из исходного датасета. Для этого проделайте следующие шаги:

1. Загрузите датасет `vehicles_dataset_old.csv`.
2. Удалите из него переменную `price` и все строковые колонки. Дерево решений и случайный лес не умеют самостоятельно работать со строковыми значениями.
3. Сформируйте x_train и x_test так же, как они были сформированы в предыдущих заданиях.
4. Обучите свою лучшую модель случайного леса на новых данных и замерьте качество. Убедитесь, что оно ухудшилось.
5. Найдите три фичи, которые лучшим образом повлияли на предсказательную способность модели.

In [40]:
df_old = pd.read_csv('data/vehicles_dataset_old.csv')
df_old.head()

,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,title_status,transmission,image_url,description,state,lat,long,posting_date,price_category,date
0,7308295377,https://chattanooga.craigslist.org/ctd/d/chatt...,chattanooga,https://chattanooga.craigslist.org,54990,2020,ram,2500 crew cab big horn,diesel,27442,clean,other,https://images.craigslist.org/00N0N_1xMPvfxRAI...,Carvana is the safer way to buy a car During t...,tn,35.060000,-85.250000,2021-04-17T12:30:50-0400,high,2021-04-17 16:30:50+00:00
1,7316380095,https://newjersey.craigslist.org/ctd/d/carlsta...,north jersey,https://newjersey.craigslist.org,16942,2016,ford,explorer 4wd 4dr xlt,other,60023,clean,automatic,https://images.craigslist.org/00x0x_26jl9F0cnL...,***Call Us for more information at: 201-635-14...,nj,40.821805,-74.061962,2021-05-03T15:40:21-0400,medium,2021-05-03 19:40:21+00:00
2,7313733749,https://reno.craigslist.org/ctd/d/atlanta-2017...,reno / tahoe,https://reno.craigslist.org,35590,2017,volkswagen,golf r hatchback,gas,14048,clean,other,https://images.craigslist.org/00y0y_eeZjWeiSfb...,Carvana is the safer way to buy a car During t...,ca,33.779214,-84.411811,2021-04-28T03:52:20-0700,high,2021-04-28 10:52:20+00:00
3,7308210929,https://fayetteville.craigslist.org/ctd/d/rale...,fayetteville,https://fayetteville.craigslist.org,14500,2013,toyota,rav4,gas,117291,clean,automatic,https://images.craigslist.org/00606_iGe5iXidib...,2013 Toyota RAV4 XLE 4dr SUV Offered by: R...,nc,35.715954,-78.655304,2021-04-17T10:08:57-0400,medium,2021-04-17 14:08:57+00:00
4,7303797340,https://knoxville.craigslist.org/ctd/d/knoxvil...,knoxville,https://knoxville.craigslist.org,14590,2012,bmw,1 series 128i coupe 2d,other,80465,clean,other,https://images.craigslist.org/00F0F_5UAXmOzC18...,Carvana is the safer way to buy a car During t...,tn,35.970000,-83.940000,2021-04-08T15:10:56-0400,medium,2021-04-08 19:10:56+00:00


In [42]:
# Ваш код здесь 
df_numeric = df_old.select_dtypes(include='number')

In [43]:
df_numeric

,id,price,year,odometer,lat,long
0,7308295377,54990,2020,27442,35.060000,-85.250000
1,7316380095,16942,2016,60023,40.821805,-74.061962
2,7313733749,35590,2017,14048,33.779214,-84.411811
3,7308210929,14500,2013,117291,35.715954,-78.655304
4,7303797340,14590,2012,80465,35.970000,-83.940000
...,...,...,...,...,...,...
9614,7304876387,4495,2002,150000,42.123900,-79.189500
9615,7316152972,14495,2008,113573,43.216990,-77.755610
9616,7310993818,8995,2011,150184,44.925908,-122.982753
9617,7306637427,31900,2015,61943,43.029559,-89.397796


In [45]:
x = df_numeric.drop(['price'], axis=1)
y = df_old['price_category']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [51]:
rfc3 = RandomForestClassifier(n_estimators = 600,
                              min_samples_split = 3,
                              min_samples_leaf = 1,
                              max_features = 'log2',
                              verbose = 1,
                              n_jobs = -1)

In [53]:
rfc3.fit(x_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:    3.0s finished


RandomForestClassifier(max_features='log2', min_samples_split=3,
                       n_estimators=600, n_jobs=-1, verbose=1)

In [54]:
rfc3_pred = rfc3.predict(x_test)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 600 out of 600 | elapsed:    0.0s finished


In [57]:
accuracy_score(y_test, rfc3_pred)

0.6386001386001386

In [58]:
confusion_matrix(y_test, rfc3_pred)

array([[709,  76, 212],
       [ 44, 706, 203],
       [272, 236, 428]], dtype=int64)

In [59]:
f_imp_list3 = list(zip(x_train.columns, rfc3.feature_importances_))
f_imp_list3.sort(key=lambda x: x[1], reverse=True)

In [64]:
f_imp_list3[:3]

[('odometer', 0.2718295658755209),
 ('year', 0.23823405550828022),
 ('id', 0.16450797128653277)]